In [ ]:
!nvidia-smi

Thu Jun  9 09:53:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q sentence-transformers

In [ ]:
import os
import sys

import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer, models
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from sentence_transformers.losses import DenoisingAutoEncoderLoss

In [ ]:
clean_legal_corpus_df = pd.read_pickle('/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/datasets/clean_legal_corpus.pkl')
clean_legal_corpus_df

text
law_id         article_id                                                   
01/2009/tt-bnn 1           phạm_vi áp_dụng thông_tư hướng_dẫn tuần_tra ca...
               2           tổ_chức lực_lượng hàng trước mùa mưa_lũ uỷ_ban...
               3           tiêu_chuẩn thành_viên lực_lượng tuần_tra canh_...
               4           nhiệm_vụ lực_lượng tuần_tra canh_gác đê chấp_h...
               5           phù_hiệu lực_lượng tuần_tra canh_gác đê phù_hi...
...                                                                      ...
99/2020/nđ-cp  60          thẩm_quyền xử_phạt hải_quan chi_cục trưởng chi...
               61          thẩm_quyền xử_phạt quản_lý thị_trường kiểm_soá...
               62          thẩm_quyền xử_phạt thanh_tra thanh_tra_viên gi...
               63          phân_định thẩm_quyền xử_phạt chủ_tịch uỷ_ban_n...
               64          hiệu_lực thi_hành nghị_định hiệu_lực thi_hành ...

[61425 rows x 1 columns]

In [ ]:
sentences = clean_legal_corpus_df['text'].tolist()
sentences[:10]

['phạm_vi áp_dụng thông_tư hướng_dẫn tuần_tra canh_gác bảo_vệ đê_điều mùa lũ đối_với tuyến đê sông phân_loại phân_cấp quy_định luật đê_điều',
 'tổ_chức lực_lượng hàng trước mùa mưa_lũ uỷ_ban_nhân_dân cấp xã đê tổ_chức lực_lượng lao_động địa_phương tuần_tra canh_gác đê thường_trực điếm_canh đê nhà dân khu_vực gần đê đối_với những khu_vực chưa điếm_canh đê báo_động lũ cấp trở lên đối_với tuyến sông đê gọi tắt lực_lượng tuần_tra canh_gác đê lực_lượng tuần_tra canh_gác đê tổ_chức thành đội uỷ_ban_nhân_dân cấp xã ra quyết_định thành_lập kilômét đê thành_lập đội mỗi đội đội_trưởng đội phó danh_sách thành_viên đội tuần_tra canh_gác đê niêm_yết điếm_canh đê địa_bàn phân_công lũ bão diễn_biến phức_tạp kéo_dài uỷ_ban_nhân_dân cấp xã có_thể quyết_định bổ_sung thêm thành_viên đội tuần_tra canh_gác đê',
 'tiêu_chuẩn thành_viên lực_lượng tuần_tra canh_gác đê khoẻ_mạnh tháo_vát đủ khả_năng đảm_đương những công_việc nặng_nhọc kể_cả lúc mưa to gió lớn đêm_tối tinh_thần trách_nhiệm chịu_đựng gian_khổ kh

In [ ]:
train_data = DenoisingAutoEncoderDataset(sentences)
train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True, drop_last=False)

In [ ]:
bert = models.Transformer('vinai/phobert-large')
pooling = models.Pooling(bert.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)

model = SentenceTransformer(modules=[bert, pooling])
model

Some weights of the model checkpoint at vinai/phobert-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at vinai/phobert-large and are newly initialized: ['roberta.encoder.layer.6.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.23.crossattention.self.key.bias', 'roberta.encoder.layer.12.crossattention.self.key.weight', 'roberta.encoder.layer.9.crossattention.output.dense.weight', 'roberta.encoder.layer.23.crossattention.self.value.weight', 'roberta.encoder.layer.5.crossattention.self.query.bias', 'roberta.encoder.layer.12.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.4.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.20.crossattention.self.key.bias', 'roberta.encoder.layer.7.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.21.crossattention.output.LayerNorm.weight', 'roberta.en

In [ ]:
# !pip install -q nltk

# import nltk
# nltk.download('punkt')

In [10]:
model.fit(
    train_objectives=[(train_dataloader, loss)],
    epochs=10,
    scheduler='warmuplinear',
    optimizer_params={'lr': 3e-5},
    use_amp=True,
    show_progress_bar=True,
    checkpoint_path='/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/checkpoints/TSDAE_legal_corpus/',
    checkpoint_save_steps=15000,
    checkpoint_save_total_limit=2
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15357 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/15357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15357 [00:00<?, ?it/s]